<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 3 - 3 Hours </h1>
<h1 style="text-align:center">Long Short Term Memory (LSTM) for Language Modeling</h1>

<b> Student 1:</b> THAN Thanh An  
<b> Student 2:</b> NGUYEN Minh Duc
 
 
In this Lab Session,  you will build and train a Recurrent Neural Network, based on Long Short-Term Memory (LSTM) units for next word prediction task. 

Answers and experiments should be made by groups of one or two students. Each group should fill and run appropriate notebook cells. 
Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an pdf document using print as PDF (Ctrl+P). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by June 9th 2017.

Send you pdf file to benoit.huet@eurecom.fr and olfa.ben-ahmed@eurecom.fr using **[DeepLearning_lab3]** as Subject of your email.

#  Introduction

You will train a LSTM to predict the next word using a sample short story. The LSTM will learn to predict the next item of a sentence from the 3 previous items (given as input). Ponctuation marks are considered as dictionnary items so they can be predicted too. Figure 1 shows the LSTM and the process of next word prediction. 

<img src="lstm.png" height="370" width="370"> 


Each word (and ponctuation) from text sentences is encoded by a unique integer. The integer value corresponds to the index of the corresponding word (or punctuation mark) in the dictionnary. The network output is a one-hot-vector indicating the index of the predicted word in the reversed dictionnary (Section 1.2). For example if the prediction is 86, the predicted word will be "company". 



You will use a sample short story from Aesop’s Fables (http://www.taleswithmorals.com/) to train your model. 


<font size="3" face="verdana" > <i> "There was once a young Shepherd Boy who tended his sheep at the foot of a mountain near a dark forest.

It was rather lonely for him all day, so he thought upon a plan by which he could get a little company and some excitement.
He rushed down towards the village calling out "Wolf, Wolf," and the villagers came out to meet him, and some of them stopped with him for a considerable time.
This pleased the boy so much that a few days afterwards he tried the same trick, and again the villagers came to his help.
But shortly after this a Wolf actually did come out from the forest, and began to worry the sheep, and the boy of course cried out "Wolf, Wolf," still louder than before.
But this time the villagers, who had been fooled twice before, thought the boy was again deceiving them, and nobody stirred to come to his help.
So the Wolf made a good meal off the boy's flock, and when the boy complained, the wise man of the village said:
"A liar will not be believed, even when he speaks the truth."  "</i> </font>.    







Start by loading the necessary libraries and resetting the default computational graph. For more details about the rnn packages, we suggest you to take a look at https://www.tensorflow.org/api_guides/python/contrib.rnn

In [1]:
import numpy as np
import collections # used to build the dictionary
import random
import time
import pickle # may be used to save your model 
import matplotlib.pyplot as plt
#Import Tensorflow and rnn
import tensorflow as tf
from tensorflow.contrib import rnn  

# Target log path
logs_path = 'lstm_words'
writer = tf.summary.FileWriter(logs_path)

# Next-word prediction task

## Part 1: Data  preparation

### 1.1. Loading data

Load and split the text of our story

In [2]:
def load_data(filename):
    with open(filename) as f:
        data = f.readlines()
    data = [x.strip().lower() for x in data]
    data = [data[i].split() for i in range(len(data))]
    data = np.array(data)
    data = np.reshape(data, [-1, ])
    print(data[:10])
    return data

#Run the cell 
train_file ='data/story.txt'
train_data = load_data(train_file)
print("Loaded training data...")
print("Traning data size: " + str(len(train_data)) + " words")

['there' 'was' 'once' 'a' 'young' 'shepherd' 'boy' 'who' 'tended' 'his']
Loaded training data...
Traning data size: 214 words


### 1.2.Symbols encoding

The LSTM input's can only be numbers. A way to convert words (symbols or any items) to numbers is to assign a unique integer to each word. This process is often based on frequency of occurrence for efficient coding purpose.

Here, we define a function to build an indexed word dictionary (word->number). The "build_vocabulary" function builds both:

- Dictionary : used for encoding words to numbers for the LSTM inputs 
- Reverted dictionnary : used for decoding the outputs of the LSTM into words (and punctuation).

For example, in the story above, we have **113** individual words. The "build_vocabulary" function builds a dictionary with the following entries ['the': 0], [',': 1], ['company': 85],...


In [3]:
def build_vocabulary(words):
    count = collections.Counter(words).most_common()
    dic= dict()
    for word, _ in count:
        dic[word] = len(dic)
    reverse_dic= dict(zip(dic.values(), dic.keys()))
    return dic, reverse_dic


Run the cell below to display the vocabulary

In [4]:
dictionary, reverse_dictionary = build_vocabulary(train_data)
vocabulary_size= len(dictionary) 
print("Dictionary size (Vocabulary size) = ", vocabulary_size)
print("\n")
print("Dictionary : \n")
print(dictionary)
print("\n")
print("Reverted Dictionary : \n" )
print(reverse_dictionary)

('Dictionary size (Vocabulary size) = ', 113)


Dictionary : 

{'all': 32, 'liar': 33, 'help': 17, 'cried': 34, 'course': 35, 'still': 36, 'pleased': 37, 'before': 18, 'excitement': 91, 'deceiving': 38, 'had': 39, 'young': 69, 'actually': 40, 'to': 6, 'villagers': 11, 'shepherd': 41, 'them': 19, 'lonely': 42, 'get': 44, 'dark': 45, 'not': 64, 'day': 47, 'did': 48, 'calling': 49, 'twice': 50, 'good': 51, 'stopped': 52, 'truth': 53, 'meal': 54, 'sheep,': 55, 'some': 20, 'tended': 56, 'louder': 57, 'flock': 58, 'out': 9, 'even': 59, 'trick': 60, 'said': 61, 'for': 21, 'be': 62, 'after': 63, 'come': 22, 'by': 65, 'boy': 7, 'of': 10, 'could': 66, 'days': 67, 'wolf': 5, 'afterwards': 68, ',': 1, 'down': 70, 'village': 23, 'sheep': 72, 'little': 73, 'from': 74, 'rushed': 75, 'there': 76, 'been': 77, '.': 4, 'few': 78, 'much': 79, "boy's": 80, ':': 81, 'was': 12, 'a': 2, 'him': 13, 'that': 83, 'company': 84, 'nobody': 85, 'but': 24, 'fooled': 86, 'with': 87, 'than': 43, 'he': 8, 'made': 89, 'w

## Part 2 : LSTM Model in TensorFlow

Since you have defined how the data will be modeled, you are now to develop an LSTM model to predict the word of following a sequence of 3 words. 

### 2.1. Model definition

Define a 2-layers LSTM model.  

For this use the following classes from the tensorflow.contrib library:

- rnn.BasicLSTMCell(number of hidden units) 
- rnn.static_rnn(rnn_cell, data, dtype=tf.float32)
- rnn.MultiRNNCell(,)


You may need some tensorflow functions (https://www.tensorflow.org/api_docs/python/tf/) :
- tf.split
- tf.reshape 
- ...




Training Parameters and constants

In [5]:
import inspect
import time
def lstm_cell(size):
    return rnn.BasicLSTMCell(
            size, state_is_tuple=True)#,reuse=tf.get_variable_scope().reuse)
    
def lstm_model(x, w, b, n_input):
    # reshape to [1, n_input]
    
    x_reshaped = tf.reshape(x, [-1, n_input])
    x_split = tf.split(x_reshaped, n_input, 1)
    
    timestamp = str(time.time())
    with tf.variable_scope('cell_def_' + timestamp):
        rnn_cell = rnn.MultiRNNCell([lstm_cell(n_hidden),lstm_cell(n_hidden)], state_is_tuple = True)

    with tf.variable_scope('rnn_def_' + timestamp):
        outputs, states = rnn.static_rnn(rnn_cell, x_split, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], w['out']) + b['out']

In [11]:
# Training Parameters
learning_rate = 0.001
epochs = 100000
display_step = 10000
n_input = 3

#For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }

beta = 0.01 # regularization coeff

#build the model
pred = lstm_model(x, weights, biases, n_input)

Define the Loss/Cost and optimizer

In [13]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y)) #+ beta*(tf.nn.l2_loss(weights["out"]) + tf.nn.l2_loss(biases["out"])) #Cross Entropy loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost) #use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

We give you here the Test Function

In [14]:
#run the cell
def test(sentence, session, verbose=False):
    sentence = sentence.strip()
    words = sentence.split(' ')
    if len(words) != n_input:
        print("sentence length should be equel to", n_input, "!")
    try:
        symbols_inputs = [dictionary[str(words[i - n_input])] for i in range(n_input)]
        keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        onehot_pred = session.run(pred, feed_dict={x: keys})
        onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
        words.append(reverse_dictionary[onehot_pred_index])
        sentence = " ".join(words)
        if verbose:
            print(sentence)
        return reverse_dictionary[onehot_pred_index]
    except:
        print(" ".join(["Word", words[n_input - i], "not in dictionary"]))

## Part 3 : LSTM Training  

In the Training process, at each epoch, 3 words are taken from the training data, encoded to integer to form the input vector. The training labels are one-hot vector encoding the word that comes after the 3 inputs words. Display the loss and the training accuracy every 1000 iteration. Save the model at the end of training in the **lstm_model** folder

In [15]:
epochs = 500000

In [16]:
def trainLSTM(train_data, optimizer, accuracy, cost, pred):
    # Initializing the variables
    start_time = time.time()
    init = tf.global_variables_initializer()
    model_saver = tf.train.Saver()
    nWords = len(train_data)
    session = tf.Session()
    session.run(init)
    step = 0
    offset = 0
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    print("Start Training")

    ##############################################
    while step < epochs:
        if offset+n_input+1 > nWords:
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys),[-1,n_input,1])


        symbols_out_onehot = np.zeros([vocabulary_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        #print(symbols_in_keys.shape,symbols_out_onehot.shape)
        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], 
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= ", str(step+1), ", Average loss= {:.4f}".format(loss_total/display_step),
                                         ", Average accuracy={:.2f}".format(100*acc_total/display_step))
            loss_total = 0
            acc_total = 0

        offset += (n_input+1)
        step +=1

    ##############################################

    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and oint your web browser to the returned link")
    ##############################################
    #save your model 
    model_saver.save(session,'lstm_model/lstm_1.ckpt')
    ##############################################
    print("Model saved")
    
trainLSTM(train_data, optimizer, accuracy, cost, pred)

Start Training
('Iter= ', '10000', ', Average loss= 2.3555', ', Average accuracy=43.42')
('Iter= ', '20000', ', Average loss= 0.6745', ', Average accuracy=82.21')
('Iter= ', '30000', ', Average loss= 0.3954', ', Average accuracy=88.07')
('Iter= ', '40000', ', Average loss= 0.2796', ', Average accuracy=90.81')
('Iter= ', '50000', ', Average loss= 0.2458', ', Average accuracy=92.15')
('Iter= ', '60000', ', Average loss= 0.1923', ', Average accuracy=93.62')
('Iter= ', '70000', ', Average loss= 0.1930', ', Average accuracy=93.25')
('Iter= ', '80000', ', Average loss= 0.1781', ', Average accuracy=93.44')
('Iter= ', '90000', ', Average loss= 0.1684', ', Average accuracy=94.41')
('Iter= ', '100000', ', Average loss= 0.1609', ', Average accuracy=94.86')
('Iter= ', '110000', ', Average loss= 0.1783', ', Average accuracy=94.83')
('Iter= ', '120000', ', Average loss= 0.1498', ', Average accuracy=95.10')
('Iter= ', '130000', ', Average loss= 0.1426', ', Average accuracy=95.35')
('Iter= ', '140000'

## Part 4 : Test your model 

### 3.1. Next word prediction

Load your model (using the model_saved variable given in the training session) and test the sentences :
- 'get a little' 
- 'nobody tried to'
- Try with other sentences using words from the stroy's vocabulary. 

In [19]:
#Your implementation goes here 
model_saver = tf.train.Saver()
sentence = " ".join([reverse_dictionary[i] for i in np.random.randint(vocabulary_size, size=(3))])
# sentence = "nobody tried to"
# sentence = "get a little"
with tf.Session() as session:
    model_saver.restore(session,'./lstm_model/lstm_1.ckpt')
    test(sentence,session,verbose=True)

considerable time day at


In [17]:
session = tf.InteractiveSession()
model_saver.restore(session,'./lstm_model/lstm_1.ckpt')
test(sentence,session,verbose=True)

get a little company


'company'

The model predicted "get a little company" which is correct.  
The model predicted "nobody tried to come" which is correct.  
When we use random word from the story "considerable time day at"

### 3.2. More fun with the Fable Writer !

You will use the RNN/LSTM model learned in the previous question to create a
new story/fable.
For this you will choose 3 words from the dictionary which will start your
story and initialize your network. Using those 3 words the RNN will generate
the next word or the story. Using the last 3 words (the newly predicted one
and the last 2 from the input) you will use the network to predict the 5
word of the story.. and so on until your story is 5 sentence long. 
Make a point at the end of your story. 
To implement that, you will use the test function. 

In [18]:
#Your implementation goes here 
session = tf.InteractiveSession()
model_saver.restore(session,'./lstm_model/lstm_1.ckpt')
words = sentence.split(" ")
countSentences = 0
countWords = 0 
COUNTMAX = 500

while countSentences < 5 and countWords < COUNTMAX:
    print (" ".join(words[-3:-1]+[words[-1]]))
    w = test(" ".join(words[-3:-1]+[words[-1]]),session)
    if (w == '.'):
        countSentences +=  1
    words.append(w)
    countWords += 1    

get a little
a little company
little company and
company and some
and some excitement
some excitement .
excitement . he
. he rushed
he rushed down
rushed down towards
down towards ,
towards , even
, even when
even when he
when he speaks
he speaks the
speaks the same
the same trick
same trick ,
trick , and
, and out
and out to
out to ,
to , wolf
, wolf ,
wolf , still
, still louder
still louder than
louder than before
than before .
before . but
. but this
but this time
this time the
time the villagers
the villagers ,
villagers , who
, who had
who had been
had been fooled
been fooled twice
fooled twice before
twice before ,
before , thought
, thought the
thought the boy
the boy was
boy was again
was again deceiving
again deceiving them
deceiving them ,
them , and
, and nobody
and nobody stirred
nobody stirred to
stirred to could
to could meal
could meal towards
meal towards ,
towards , even
, even when
even when he
when he speaks
he speaks the
speaks the same
the same trick
same trick ,


In [19]:
" ".join(words)

'get a little company and some excitement . he rushed down towards , even when he speaks the same trick , and out to , wolf , still louder than before . but this time the villagers , who had been fooled twice before , thought the boy was again deceiving them , and nobody stirred to could meal towards , even when he speaks the same trick , and out to , wolf , still louder than before . but this time the villagers , who had been fooled twice before , thought the boy was again deceiving them , and nobody stirred to could meal towards , even when he speaks the same trick , and out to , wolf , still louder than before . but this time the villagers , who had been fooled twice before , thought the boy was again deceiving them , and nobody stirred to could meal towards , even when he speaks the same trick , and out to , wolf , still louder than before .'

The generated text seems to be converging to a single sentence and repeat it. This may be due to our small dataset in term of training sequence and vocabulary. We can try adding more data to train the model.

### 3.3. Play with number of inputs

The number of input in our example is 3, see what happens when you use other number (1 and 5)

In [21]:
# Training Parameters
learning_rate = 0.001
epochs = 500000
display_step = 1000
n_input = 5

#For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }

beta = 0.01 # regularization coeff

#build the model
pred = lstm_model(x, weights, biases, n_input)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost) #use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

trainLSTM(train_data, optimizer, accuracy, cost, pred)


Start Training
('Iter= ', '1000', ', Average loss= 4.3338', ', Average accuracy=8.40')
('Iter= ', '2000', ', Average loss= 3.6666', ', Average accuracy=14.30')
('Iter= ', '3000', ', Average loss= 2.9087', ', Average accuracy=32.80')
('Iter= ', '4000', ', Average loss= 2.3014', ', Average accuracy=43.30')
('Iter= ', '5000', ', Average loss= 1.5205', ', Average accuracy=62.90')
('Iter= ', '6000', ', Average loss= 1.1214', ', Average accuracy=71.00')
('Iter= ', '7000', ', Average loss= 0.6827', ', Average accuracy=82.70')
('Iter= ', '8000', ', Average loss= 0.4769', ', Average accuracy=87.20')
('Iter= ', '9000', ', Average loss= 0.3088', ', Average accuracy=92.30')
('Iter= ', '10000', ', Average loss= 0.1841', ', Average accuracy=95.50')
('Iter= ', '11000', ', Average loss= 0.1790', ', Average accuracy=96.00')
('Iter= ', '12000', ', Average loss= 0.1481', ', Average accuracy=96.10')
('Iter= ', '13000', ', Average loss= 0.0809', ', Average accuracy=97.70')
('Iter= ', '14000', ', Average lo

In [22]:
# Training Parameters
learning_rate = 0.001
epochs = 500000
display_step = 1000
n_input = 1

#For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }

beta = 0.01 # regularization coeff

#build the model
pred = lstm_model(x, weights, biases, n_input)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost) #use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

trainLSTM(train_data, optimizer, accuracy, cost, pred)


Start Training
('Iter= ', '1000', ', Average loss= 4.7021', ', Average accuracy=5.30')
('Iter= ', '2000', ', Average loss= 4.3234', ', Average accuracy=7.30')
('Iter= ', '3000', ', Average loss= 4.1402', ', Average accuracy=10.30')
('Iter= ', '4000', ', Average loss= 4.0838', ', Average accuracy=11.60')
('Iter= ', '5000', ', Average loss= 3.9541', ', Average accuracy=12.10')
('Iter= ', '6000', ', Average loss= 3.8557', ', Average accuracy=12.00')
('Iter= ', '7000', ', Average loss= 3.8615', ', Average accuracy=10.80')
('Iter= ', '8000', ', Average loss= 3.9697', ', Average accuracy=11.30')
('Iter= ', '9000', ', Average loss= 3.6916', ', Average accuracy=13.30')
('Iter= ', '10000', ', Average loss= 3.9909', ', Average accuracy=11.00')
('Iter= ', '11000', ', Average loss= 3.8846', ', Average accuracy=11.50')
('Iter= ', '12000', ', Average loss= 3.8591', ', Average accuracy=12.70')
('Iter= ', '13000', ', Average loss= 3.5415', ', Average accuracy=14.70')
('Iter= ', '14000', ', Average los

In [24]:
# Training Parameters
learning_rate = 0.001
epochs = 100000
display_step = 100000
n_input = 1

#For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

for ni in np.arange(1, 10, 2):
    n_input = ni
    # tf Graph input
    x = tf.placeholder("float", [None, n_input, 1])
    y = tf.placeholder("float", [None, vocabulary_size])

    # LSTM  weights and biases
    weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
    biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }

    beta = 0.01 # regularization coeff

    #build the model
    pred = lstm_model(x, weights, biases, n_input)

    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost) #use RMSProp Optimizer

    # Model evaluation
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    trainLSTM(train_data, optimizer, accuracy, cost, pred)


Start Training
('Iter= ', '100000', ', Average loss= 3.6677', ', Average accuracy=14.17')
End Of training Finished!
('time: ', 127.26592493057251)
For tensorboard visualisation run on command line.
	tensorboard --logdir=lstm_words
and oint your web browser to the returned link
Model saved
Start Training
('Iter= ', '100000', ', Average loss= 0.5127', ', Average accuracy=85.98')
End Of training Finished!
('time: ', 132.43285083770752)
For tensorboard visualisation run on command line.
	tensorboard --logdir=lstm_words
and oint your web browser to the returned link
Model saved
Start Training
('Iter= ', '100000', ', Average loss= 0.2160', ', Average accuracy=94.77')
End Of training Finished!
('time: ', 164.6172218322754)
For tensorboard visualisation run on command line.
	tensorboard --logdir=lstm_words
and oint your web browser to the returned link
Model saved
Start Training
('Iter= ', '100000', ', Average loss= 0.1631', ', Average accuracy=96.39')
End Of training Finished!
('time: ', 201.

The number of input = 5 yield superior result to 3 and 1 because we have more feature to predict the next word. If we increase the number of input the result may be better but if it is too big the model will overfit with the training data.